In [ ]:
# Get credentials
from IPython.utils import io
with io.capture_output() as captured:
    %run ../Introduction.ipynb
    
from datetime import datetime, timedelta
from sentenai import Sentenai
sentenai = Sentenai(host=host, port=port)

# Introduction to TSPL